In [6]:


#%load_ext line_profiler

#%lprun -f __main__ __main__()
import math
import numpy as np

#from pydrake.all import (BasicVector, DiagramBuilder, FloatingBaseType,
                         #RigidBodyPlant, RigidBodyTree, Simulator,SignalLogger)

from pydrake.all import *
from underactuated import (FindResource, PlanarRigidBodyVisualizer)

from IPython.display import HTML


tree = RigidBodyTree(FindResource("../../notebooks/walkers/three_link.urdf"), FloatingBaseType.kFixed)


box_depth = 100
pi = math.pi

X = Isometry3(rotation=np.identity(3), translation=[0, 0, -5.])
                
color = np.array([0.9297, 0.7930, 0.6758, 1])
tree.world().AddVisualElement(VisualElement(Box([100., 1., 10.]), X, color))
tree.addCollisionElement(CollisionElement(Box([100., 1., 10.]), X), tree.world(), "the_ground")
tree.compile()

plant = RigidBodyPlant(tree)
context = plant.CreateDefaultContext()

dircol = DirectCollocation(plant, context, num_time_samples=21,
                           minimum_timestep=0.01, maximum_timestep=0.01)

dircol.AddEqualTimeIntervalsConstraints()


initial_state_neg  = (pi - pi/8., pi + pi/4. ,3*pi/2. - pi/4, -50.,-50.,-50.)
initial_state_pos  = (pi - pi/8., pi + pi/4. ,3*pi/2. - pi/4,  50., 50., 50.)
initial_state_zero = (pi - pi/8., pi + pi/4. ,3*pi/2. - pi/4,  0., 0., 0.)

#dircol.AddBoundingBoxConstraint(initial_state_neg, initial_state_pos,
#                                dircol.initial_state())

final_state_neg  = (pi + pi/8, pi - pi/4  , 3*pi/2 - pi/4,-50.,-50.,-50.)
final_state_pos  = (pi + pi/8, pi - pi/4  , 3*pi/2 - pi/4, 50., 50., 50.)
final_state_zero = (pi + pi/8, pi - pi/4  , 3*pi/2 - pi/4,  0.,  0.,  0.)

dircol.AddBoundingBoxConstraint(final_state_neg, final_state_pos,
                                dircol.final_state())

#dircol.AddBoundingBoxConstraint(dircol.initial_state(), dircol.initial_state(),
#                                dircol.final_state())

#dircol.AddConstraint()


R = 10  # Cost on input "effort".
u = dircol.input()

dircol.AddRunningCost(R*u[0]**2)
dircol.AddRunningCost(R*u[1]**2)

dircol.AddFinalCost(dircol.time())


initial_x_trajectory = PiecewisePolynomial.FirstOrderHold([0., 4.],
                                       np.column_stack((initial_state_zero,
                                                        final_state_zero)))

dircol.SetInitialTrajectory(PiecewisePolynomial(), initial_x_trajectory)



dircol.AddLinearConstraint(dircol.final_state()[0] == pi - dircol.initial_state()[0])
dircol.AddLinearConstraint(dircol.final_state()[1] == - dircol.initial_state()[1])
#dircol.AddLinearConstraint(dircol.final_state()[2] + dircol.final_state()[1] == dircol.initial_state()[2] + dircol.initial_state()[0])


dircol.AddLinearConstraint(dircol.final_state()[3] == -dircol.initial_state()[3])
dircol.AddLinearConstraint(dircol.final_state()[4] == -dircol.initial_state()[4])
#dircol.AddLinearConstraint(dircol.final_state()[5] == dircol.initial_state()[5])


xh_fn = cos(dircol.final_state()[0])
xe_fn = xh_fn + cos(dircol.final_state()[0] + dircol.final_state()[1])
xt_fn = xh_fn + cos(dircol.final_state()[0] + dircol.final_state()[2])

xh_in = cos(dircol.initial_state()[0])
xe_in = xh_in + cos(dircol.initial_state()[0] + dircol.initial_state()[1])
xt_in = xh_in + cos(dircol.initial_state()[0] + dircol.initial_state()[2])

dircol.AddConstraint(xt_fn >= .1)
dircol.AddConstraint(xt_in <= -.1)


dircol.SetSolverOption()
result = dircol.Solve()
assert(result == SolutionResult.kSolutionFound)

x_trajectory = dircol.ReconstructStateTrajectory()


u_trajectory = dircol.ReconstructInputTrajectory()
times = np.linspace(u_trajectory.start_time(), u_trajectory.end_time(), 100)
u_lookup = np.vectorize(u_trajectory.value)

u_values = np.zeros((times.size,2),float)
for i,t in enumerate(times):
    u_values[i,:] = np.transpose(u_lookup(t))
    
plt.figure()
plt.plot(times, u_values)
plt.xlabel('time (seconds)')
plt.ylabel('force (Newtons)')


prbv = PlanarRigidBodyVisualizer(tree, xlim=[-2.5, 2.5], ylim=[-1, 2.5])
ani = prbv.animate(x_trajectory, resample=100, repeat=True)
plt.close(prbv.fig)
HTML(ani.to_html5_video())
    
    

AssertionError: 

In [3]:
#help(u_lookup)
print u_trajectory.value(0)
print u_lookup
#help(u_trajectory.value)
#help(prbv.animate)
help(dircol.AddBoundingBoxConstraint)


[[ 0.00014389]
 [-0.00306981]]
Help on method AddBoundingBoxConstraint in module pydrake.solvers._mathematicalprogram_py:

AddBoundingBoxConstraint(...) method of pydrake.systems.trajectory_optimization.DirectCollocation instance
    AddBoundingBoxConstraint(*args, **kwargs)
    Overloaded function.
    
    1. AddBoundingBoxConstraint(self: pydrake.solvers._mathematicalprogram_py.MathematicalProgram, arg0: numpy.ndarray[float64[m, 1]], arg1: numpy.ndarray[float64[m, 1]], arg2: numpy.ndarray[object[m, 1]]) -> drake::solvers::Binding<drake::solvers::BoundingBoxConstraint>
    
    2. AddBoundingBoxConstraint(self: pydrake.solvers._mathematicalprogram_py.MathematicalProgram, arg0: float, arg1: float, arg2: pydrake._symbolic_py.Variable) -> drake::solvers::Binding<drake::solvers::BoundingBoxConstraint>
    
    3. AddBoundingBoxConstraint(self: pydrake.solvers._mathematicalprogram_py.MathematicalProgram, arg0: float, arg1: float, arg2: numpy.ndarray[object[m, n], flags.f_contiguous]) -> d

In [45]:
u_values = np.zeros((times.size,2),float)
for i,t in enumerate(times):
    u_values[i,:] = np.transpose(u_lookup(t))


In [ ]:
print "hello?"